In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import pca_grid_search

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[3]
dataset_name = datasets[1]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/twitter/pca/me2\_[dimensiones]\_[dominio\_fuente]_[dominio\_objetivo].pkl

# Pruebas con el dataset de Twitter (2000 Dimensiones)

In [3]:
dims = 2000

In [4]:
print tipo
print dataset_name
print dims
print data_path

pca
twitter
2000
data


In [5]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [6]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'n_components': [int(dims/2), int(dims/3), int(dims/4)],
}

pairs = list(itertools.permutations(domains, 2))
i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%d) %s - %s" % (i, src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()

    X_train = np.concatenate((X_src, X_tgt))

    best_model, score = pca_grid_search(X_train, X_src, y_src, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s\n" % best_model_save_path
    joblib.dump(best_model, best_model_save_path)
    i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
1) rio2016 - thevoice
	n_components: 1000
	0.890

	n_components: 666
	0.885

	n_components: 500
	0.885

	Mejor modelo: 0.890
	Guardando modelo en models/twitter/pca/me2_2000_rio2016_thevoice.pkl

2) rio2016 - general
	n_components: 1000
	0.890

	n_components: 666
	0.886

	n_components: 500
	0.875

	Mejor modelo: 0.890
	Guardando modelo en models/twitter/pca/me2_2000_rio2016_general.pkl

3) thevoice - rio2016
	n_components: 1000
	0.901

	n_components: 666
	0.898

	n_components: 500
	0.899

	Mejor modelo: 0.901
	Guardando modelo en models/twitter/pca/me2_2000_thevoice_rio2016.pkl

4) thevoice - general
	n_components: 1000
	0.901

	n_components: 666
	0.893

	n_components: 500
	0.886

	Mejor modelo: 0.901
	Guardando modelo en models/twitter/pca/me2_2000_thevoice_general.pkl

5) general - rio2016
	n_components: 1000


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.709

	n_components: 666


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.715

	n_components: 500
	0.710

	Mejor modelo: 0.715
	Guardando modelo en models/twitter/pca/me2_2000_general_rio2016.pkl

6) general - thevoice
	n_components: 1000
	0.712

	n_components: 666
	0.712

	n_components: 500
	0.712

	Mejor modelo: 0.712
	Guardando modelo en models/twitter/pca/me2_2000_general_thevoice.pkl


Adaptaciones completadas


## Pruebas de clasificación

In [7]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### PCA ####
    #############
    # se adaptan los dominios usando PCA
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    best_pca = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = best_pca.transform(X_tr)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = best_pca.transform(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [8]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,PCA,r->t,rio2016,thevoice,7.362689,20.880682,13.517992
1,PCA,r->g,rio2016,general,26.998127,43.102032,16.103904
2,PCA,t->r,thevoice,rio2016,10.208711,13.747731,3.539020
3,PCA,t->g,thevoice,general,26.998127,40.417665,13.419538
4,PCA,g->r,general,rio2016,10.208711,19.237750,9.029038
5,PCA,g->t,general,thevoice,7.362689,21.330492,13.967803


In [9]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/pca/me2_2000.csv
Resultados guardados.


# Pruebas con el dataset de Twitter (1000 Dimensiones)

In [10]:
dims = 1000

In [11]:
print tipo
print dataset_name
print dims
print data_path

pca
twitter
1000
data


In [12]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [13]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'n_components': [int(dims/2), int(dims/3), int(dims/4)],
}

pairs = list(itertools.permutations(domains, 2))
i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%d) %s - %s" % (i, src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()

    X_train = np.concatenate((X_src, X_tgt))

    best_model, score = pca_grid_search(X_train, X_src, y_src, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s\n" % best_model_save_path
    joblib.dump(best_model, best_model_save_path)
    i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
1) rio2016 - thevoice
	n_components: 500
	0.883

	n_components: 333
	0.879

	n_components: 250
	0.872

	Mejor modelo: 0.883
	Guardando modelo en models/twitter/pca/me2_1000_rio2016_thevoice.pkl

2) rio2016 - general
	n_components: 500
	0.885

	n_components: 333
	0.864

	n_components: 250
	0.866

	Mejor modelo: 0.885
	Guardando modelo en models/twitter/pca/me2_1000_rio2016_general.pkl

3) thevoice - rio2016
	n_components: 500
	0.856

	n_components: 333
	0.849

	n_components: 250
	0.843

	Mejor modelo: 0.856
	Guardando modelo en models/twitter/pca/me2_1000_thevoice_rio2016.pkl

4) thevoice - general
	n_components: 500
	0.849

	n_components: 333
	0.842

	n_components: 250
	0.833

	Mejor modelo: 0.849
	Guardando modelo en models/twitter/pca/me2_1000_thevoice_general.pkl

5) general - rio2016
	n_components: 500
	0.661

	n_components: 333
	0.663

	n_components: 250
	0.651

	Mejor modelo: 0.663
	Guardando modelo en models/twitter/pca/me2_1000_general_rio2016.pkl

6) general -

## Pruebas de clasificación

In [14]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### PCA ####
    #############
    # se adaptan los dominios usando PCA
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    best_pca = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = best_pca.transform(X_tr)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = best_pca.transform(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [15]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,PCA,r->t,rio2016,thevoice,19.045928,22.608902,3.562973
1,PCA,r->g,rio2016,general,30.407980,44.041652,13.633672
2,PCA,t->r,thevoice,rio2016,7.372958,19.033575,11.660617
3,PCA,t->g,thevoice,general,30.407980,43.552563,13.144583
4,PCA,g->r,general,rio2016,7.372958,15.721416,8.348457
5,PCA,g->t,general,thevoice,19.045928,23.958333,4.912405


In [16]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/pca/me2_1000.csv
Resultados guardados.
